In [7]:
#import tensorflow as tf
import tensorflow as tf

g = tf.Graph()
with g.as_default():
#    init = tf.Variable()
    # tf Graph Input
    fixed_img = tf.placeholder("float", shape=(
        1, None, None, 1), name='FixedImage')
    moving_img = tf.placeholder("float", shape=(
        1, None, None, 1), name='MovingImage')
    # Initialize the variables (i.e. assign their default value)

    with tf.name_scope('transform_parameters'):  # Set transform parameters
        x_offset = tf.Variable(0.0, name="x_offset")
        y_offset = tf.Variable(0.0, name="y_offset")
        # we keep scale and rotation fixed
        scale = tf.placeholder("float", shape=tuple(), name="scale")
        rotation = tf.placeholder("float", shape=tuple(), name="rotation")

    with tf.name_scope('transformer_and_interpolator'):
        trans_tensor= tf.keras.preprocessing.image.apply_affine_transform(
        moving_img,
        theta=rotation,
        tx=x_offset,
        ty=y_offset)
        
    with tf.name_scope('metric'): 
        mse = tf.reduce_mean(
            tf.square(fixed_img-trans_tensor), name='MeanSquareError')
        optimizer = tf.train.GradientDescentOptimizer(1e-5).minimize(mse)

AttributeError: module 'tensorflow' has no attribute 'placeholder'

In [ ]:
flat_mat = tf.tile([tf.cos(rotation), -tf.sin(rotation), x_offset,
                            tf.sin(rotation), tf.cos(rotation), y_offset], (1,))
        flat_mat = tf.reshape(flat_mat, (1, 6))
        trans_tensor = affine_transform(moving_img, flat_mat)

In [ ]:
import numpy as np
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add()
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>" % size
    return strip_def


def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script src="//cdnjs.cloudflare.com/ajax/libs/polymer/0.3.3/platform.js"></script>
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))


show_graph(g)